In [2]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
%matplotlib inline 
import os
import gc
path = "/home/ec2-user/Sanjay/main/credit_risk/data/competitions/home-credit-default-risk/"
import lightgbm as lgbm
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt

### bureau_balance

In [9]:
buro_bal = pd.read_csv(path + 'bureau_balance.csv')
buro_bal = pd.concat([buro_bal, pd.get_dummies(buro_bal.STATUS, prefix='buro_bal_status')], axis=1).drop('STATUS', axis=1)
print(buro_bal.shape)

buro_counts = buro_bal[['SK_ID_BUREAU', 'MONTHS_BALANCE']].groupby('SK_ID_BUREAU').count()
buro_bal['buro_count'] = buro_bal['SK_ID_BUREAU'].map(buro_counts['MONTHS_BALANCE'])
avg_buro_bal = buro_bal.groupby('SK_ID_BUREAU').mean()
avg_buro_bal.columns = ['avg_buro_' + f_ for f_ in avg_buro_bal.columns]
avg_buro_bal.reset_index().to_csv(path + 'features/' + 'avg_buro_bal.csv', index=False)
del buro_bal
gc.collect(), avg_buro_bal.shape

(27299925, 10)


(94, (817395, 10))

### bureau

In [10]:
buro = pd.read_csv(path + 'bureau.csv')
buro_credit_active_dum = pd.get_dummies(buro.CREDIT_ACTIVE, prefix='ca_')
buro_credit_currency_dum = pd.get_dummies(buro.CREDIT_CURRENCY, prefix='cu_')
buro_credit_type_dum = pd.get_dummies(buro.CREDIT_TYPE, prefix='ty_')
buro_full = pd.concat([buro, buro_credit_active_dum, buro_credit_currency_dum, buro_credit_type_dum], axis=1)

buro_full = buro_full.merge(right=avg_buro_bal.reset_index(), how='left', on='SK_ID_BUREAU', suffixes=('', '_bur_bal'))
nb_bureau_per_curr = buro_full[['SK_ID_CURR', 'SK_ID_BUREAU']].groupby('SK_ID_CURR').count()
buro_full['SK_ID_BUREAU'] = buro_full['SK_ID_CURR'].map(nb_bureau_per_curr['SK_ID_BUREAU'])
avg_buro = buro_full.groupby('SK_ID_CURR').mean()
avg_buro.reset_index().to_csv(path + 'features/' + 'avg_buro.csv', index=False)
del buro_credit_active_dum, buro_credit_currency_dum, buro_credit_type_dum, buro, buro_full
gc.collect(), avg_buro.shape

(105, (305811, 46))

### previous_application

In [11]:
prev = pd.read_csv(path + 'previous_application.csv')
prev_cat_features = [f_ for f_ in prev.columns if prev[f_].dtype == 'object']
prev_dum = pd.DataFrame()
for f_ in prev_cat_features:
        prev_dum = pd.concat([prev_dum, pd.get_dummies(prev[f_], prefix=f_).astype(np.uint8)], axis=1)
        
prev = pd.concat([prev, prev_dum], axis=1)
nb_prev_per_curr = prev[['SK_ID_CURR', 'SK_ID_PREV']].groupby('SK_ID_CURR').count()
prev['SK_ID_PREV'] = prev['SK_ID_CURR'].map(nb_prev_per_curr['SK_ID_PREV'])

avg_prev = prev.groupby('SK_ID_CURR').mean()
avg_prev.reset_index().to_csv(path + 'features/' + 'avg_prev.csv', index=False)
del prev, prev_dum
gc.collect(), avg_prev.shape

(63, (338857, 163))

### POS_CASH_balance

In [12]:
pos = pd.read_csv(path + 'POS_CASH_balance.csv')
pos = pd.concat([pos, pd.get_dummies(pos['NAME_CONTRACT_STATUS'])], axis=1)
nb_prevs = pos[['SK_ID_CURR', 'SK_ID_PREV']].groupby('SK_ID_CURR').count()
pos['SK_ID_PREV'] = pos['SK_ID_CURR'].map(nb_prevs['SK_ID_PREV'])
avg_pos = pos.groupby('SK_ID_CURR').mean()
avg_pos.reset_index().to_csv(path + 'features/' + 'avg_pos.csv', index=False)
del pos, nb_prevs
gc.collect(), avg_pos.shape

(79, (337252, 15))

### credit_card_balance

In [13]:
cc_bal = pd.read_csv(path + 'credit_card_balance.csv')
cc_bal = pd.concat([cc_bal, pd.get_dummies(cc_bal['NAME_CONTRACT_STATUS'], prefix='cc_bal_status_')], axis=1)
nb_prevs = cc_bal[['SK_ID_CURR', 'SK_ID_PREV']].groupby('SK_ID_CURR').count()
cc_bal['SK_ID_PREV'] = cc_bal['SK_ID_CURR'].map(nb_prevs['SK_ID_PREV'])
avg_cc_bal = cc_bal.groupby('SK_ID_CURR').mean()
avg_cc_bal.columns = ['cc_bal_' + f_ for f_ in avg_cc_bal.columns]
avg_cc_bal.reset_index().to_csv(path + 'features/' + 'avg_cc_bal.csv', index=False)
del cc_bal, nb_prevs
gc.collect(), avg_cc_bal.shape

(71, (103558, 28))

### installments_payments

In [14]:
inst = pd.read_csv(path + 'installments_payments.csv')
nb_prevs = inst[['SK_ID_CURR', 'SK_ID_PREV']].groupby('SK_ID_CURR').count()
inst['SK_ID_PREV'] = inst['SK_ID_CURR'].map(nb_prevs['SK_ID_PREV'])
avg_inst = inst.groupby('SK_ID_CURR').mean()
avg_inst.columns = ['inst_' + f_ for f_ in avg_inst.columns]
avg_inst.reset_index().to_csv(path + 'features/' + 'avg_inst.csv', index=False)
del inst, nb_prevs
gc.collect(), avg_inst.shape

(58, (339587, 7))

### Train and Test Data

In [150]:
data = pd.read_csv(path + 'application_train.csv')
test = pd.read_csv(path + 'application_test.csv')
sample_submission = pd.read_csv(path + 'sample_submission.csv')
y = data['TARGET']
del data['TARGET']
print('Shapes : ', data.shape, test.shape, sample_submission.shape)

Shapes :  (307511, 121) (48744, 121) (48744, 2)


In [151]:
categorical_feats = [f for f in data.columns if data[f].dtype == 'object']
for f_ in categorical_feats:
    data[f_], indexer = pd.factorize(data[f_])
    test[f_] = indexer.get_indexer(test[f_])

In [152]:
data = data.merge(right=avg_buro.reset_index(), how='left', on='SK_ID_CURR')  
data = data.merge(right=avg_prev.reset_index(), how='left', on='SK_ID_CURR')
data = data.merge(right=avg_pos.reset_index(), how='left', on='SK_ID_CURR')
data = data.merge(right=avg_cc_bal.reset_index(), how='left', on='SK_ID_CURR')
data = data.merge(right=avg_inst.reset_index(), how='left', on='SK_ID_CURR')

test = test.merge(right=avg_buro.reset_index(), how='left', on='SK_ID_CURR') 
test = test.merge(right=avg_prev.reset_index(), how='left', on='SK_ID_CURR')
test = test.merge(right=avg_pos.reset_index(), how='left', on='SK_ID_CURR')
test = test.merge(right=avg_cc_bal.reset_index(), how='left', on='SK_ID_CURR')
test = test.merge(right=avg_inst.reset_index(), how='left', on='SK_ID_CURR')
gc.collect(), data.shape, test.shape

(658, (307511, 380), (48744, 380))

### Model

In [153]:
parameters = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'binary_logloss',
    'num_leaves': 31,
    'learning_rate': 0.01,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 50
}
iteration = 5000
vrbse = 400

In [154]:
def lgbm_model(X_tr, y_tr, X_va, y_va, fold, test):
    tr_data = lgb.Dataset(X_tr, label=y_tr, categorical_feature=categorical_feats)
    va_data = lgb.Dataset(X_va, label=y_va, categorical_feature=categorical_feats, reference=tr_data)
    
    model = lgb.train(parameters,
                      tr_data,
                      valid_sets=va_data,
                      num_boost_round=iteration,
                      early_stopping_rounds=120,
                      verbose_eval=vrbse)
    
    test_pred = model.predict(test)
    test_final = pd.DataFrame(test_pred, columns=['prob'+str(fold)])
    
    vald_pred = model.predict(X_va)
    y_va = y_va.reset_index(drop=True)
    valid = pd.DataFrame(vald_pred, columns=['prob'])
    valid_final = valid.join(y_va)
    
    return test_final, valid_final


In [155]:
from sklearn.model_selection import KFold
data1 = data.drop(['SK_ID_CURR'], axis=1)
test1 = test.drop(['SK_ID_CURR'], axis=1)
X = data1
kf = KFold(n_splits=10, shuffle=True, random_state=2017)
kf.get_n_splits(X), kf.split(X)

test_final = sample_submission[['SK_ID_CURR']]
oof_data = pd.DataFrame(columns=['SK_ID_CURR', 'TARGET', 'prob'])

fold=1
for train_idx, test_idx in kf.split(X):
    print("FOLD ", fold, " -----------------------------")
    X_tr = X[X.index.isin(train_idx)]
    y_tr = y[y.index.isin(train_idx)]
    X_va = X[X.index.isin(test_idx)]
    y_va = y[y.index.isin(test_idx)]
    valid_final = data[['SK_ID_CURR']][data.index.isin(test_idx)].reset_index(drop=True)
    print(X_tr.shape, X_va.shape, y_tr.shape, y_va.shape, oof_data.shape)
    
    test, valid = lgbm_model(X_tr, y_tr, X_va, y_va, fold, test1)
    
    test_final = test_final.join(test)
    oof_data = oof_data.append(valid_final.join(valid))
    
    print(test_final.shape, test.shape, oof_data.shape, valid.shape)
    fold = fold+1

FOLD  1  -----------------------------
(276759, 379) (30752, 379) (276759,) (30752,) (0, 3)


/home/ec2-user/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1036: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/home/ec2-user/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:681: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


Training until validation scores don't improve for 120 rounds.
[400]	valid_0's binary_logloss: 0.384871
[800]	valid_0's binary_logloss: 0.294381
[1200]	valid_0's binary_logloss: 0.264707
[1600]	valid_0's binary_logloss: 0.253976
[2000]	valid_0's binary_logloss: 0.249207
[2400]	valid_0's binary_logloss: 0.246507
[2800]	valid_0's binary_logloss: 0.244699
[3200]	valid_0's binary_logloss: 0.243406
[3600]	valid_0's binary_logloss: 0.242476
[4000]	valid_0's binary_logloss: 0.241766
[4400]	valid_0's binary_logloss: 0.241195
[4800]	valid_0's binary_logloss: 0.240783
Did not meet early stopping. Best iteration is:
[5000]	valid_0's binary_logloss: 0.240589
(48744, 2) (48744, 1) (30752, 3) (30752, 2)
FOLD  2  -----------------------------
(276760, 379) (30751, 379) (276760,) (30751,) (30752, 3)
Training until validation scores don't improve for 120 rounds.
[400]	valid_0's binary_logloss: 0.385574
[800]	valid_0's binary_logloss: 0.295404
[1200]	valid_0's binary_logloss: 0.265904
[1600]	valid_0's b

In [156]:
test_final['TARGET'] = test_final.iloc[:,1:].apply(lambda x : x.mean(), axis=1)

In [157]:
test_final[['SK_ID_CURR', 'TARGET']].to_csv(path + 'submit/' + 'lgbm3.csv', index=False)
oof_data.to_csv(path + 'submit/' + 'oof3.csv', index=False)

In [20]:
# test_final.head()

### Ensemble

In [25]:
import pandas as pd, copy
data1 = pd.read_csv(path + 'submit/' + 'WEIGHT_AVERAGE_RANK.csv')
data2 = pd.read_csv(path + 'submit/' + 'submission_with_selected_features.csv')
# data = data1.copy()
# data['TARGET'] = data1['TARGET']*0.7 + data2['TARGET']*0.3
# data.to_csv(path + 'submit/' +'check1.csv', index=False)

In [26]:
data2['rank'] = data.TARGET.rank(method='min')
data2['TARGET'] = (data2['rank'] - data2['rank'].min()) / (data2['rank'].max() - data2['rank'].min())

In [30]:
data = data1.copy()
data['TARGET'] = data1['TARGET']*0.8 + data2['TARGET']*0.2
data.to_csv(path + 'submit/' +'check1.csv', index=False)

SK_ID_CURR
TARGET
